AUTHOR: [@sparshsah](github.com/sparshsah)

# BOND STATS

I propose the following approximation: `irr` $\approx$ `coupon - loss - premium / wal`, where
* `irr` is internal rate of return (annualized);
* `coupon` is coupon fraction (annualized);
* `loss` is (gross-of-recoveries) loss rate (annualized);
* `premium` is the premium (if positive) or discount (if negative) paid, in percentage points; and
* `wal` is weighted-average life.

For simplicity, I neglect recoveries, which makes net-of-recoveries loss rate the same as `loss`. WLOG, I assume initial principal is \$1 and payments are made at the end of each year (this is WLOG because instead of an Earth year, you can make the unit of time anything you want, and call that your year).

I'd like to know how good this approximation is under various
* Amortization schedules
* Coupons
* Loss rates
* Premia
* Terms.

In [1]:
from typing import Final

import numpy as np
import pandas as pd

In [2]:
PRIN: Final[float] = 1.00

In [40]:
def _get_baseline_prin_pmts(
    term: int = 1,
    amort: bool = False,
) -> pd.Series:
    """Stream of repayments, assuming no defaults.
    Zero-indexed, where t=0 represents outlay.
    """
    if amort:
        pmt = PRIN / term
        pmts = pd.Series(
            [-1]
            + [pmt] * term
        )   
    else:
        pmts = pd.Series(
            [-1]
            + [0] * (term-1)
            + [PRIN]
        ) 
    # validate that we get exactly repaid
    assert np.isclose(pmts.sum(),0), pmts
    return pmts

def _get_preloss_eop_bal(
    baseline_prin_pmts: pd.Series,
) -> pd.Series:
    cum_prin_pmts = pd.concat(
        [
            pd.Series([0]),
            baseline_prin_pmts.loc[1:].cumsum(),
        ],
        verify_integrity=True,
    )
    bal = PRIN - cum_prin_pmts
    assert np.isclose(bal.iloc[-1],0), bal
    return bal

def _get_chargeoffs(
    preloss_eop_bal: pd.Series,
    loss_rate: float = 0.00,
) -> pd.Series:
    return preloss_eop_bal * loss_rate

def __get_postloss_eop_bal(
    preloss_eop_bal: pd.Series,
    chargeoffs: pd.Series,
) -> pd.Series:
    postloss_eop_bal = preloss_eop_bal - chargeoffs.cumsum()
    assert np.isclose(postloss_eop_bal.iloc[-1], 0), postloss_eop_bal
    return postloss_eop_bal

def _get_postloss_eop_bal(
    preloss_eop_bal: pd.Series,
    loss_rate: float = 0,
) -> pd.Series:
    chargeoffs = _get_chargeoffs(
        preloss_eop_bal=preloss_eop_bal,
        loss_rate=loss_rate,
    )
    postloss_eop_bal = __get_postloss_eop_bal(
        preloss_eop_bal=preloss_eop_bal,
        chargeoffs=chargeoffs,
    )
    assert np.isclose(postloss_eop_bal.iloc[-1], 0), postloss_eop_bal
    return postloss_eop_bal

def _get_cpn_pmts(
    bop_bal: pd.Series,
    cpn_frac: float = 0.00,
) -> None:
    # coupon paid on beginning-of-period balance, which at t=0 should be null
    assert np.isnan(bop_bal.loc[0]), bop_bal
    pmts = bop_bal * cpn_frac
    # fill in the null
    pmts.loc[0] = 0
    return pmts

In [41]:
baseline_prin_pmts = _get_baseline_prin_pmts(
    5,
    0,
)
baseline_prin_pmts

0   -1.0
1    0.0
2    0.0
3    0.0
4    0.0
5    1.0
dtype: float64

In [42]:
preloss_eop_bal = _get_preloss_eop_bal(
    baseline_prin_pmts=baseline_prin_pmts,
)
preloss_eop_bal

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    0.0
dtype: float64

In [43]:
chargeoffs = _get_chargeoffs(
    preloss_eop_bal=preloss_eop_bal,
    loss_rate=0.10,
)
chargeoffs

0    0.1
1    0.1
2    0.1
3    0.1
4    0.1
5    0.0
dtype: float64

In [45]:
preloss_eop_bal

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    0.0
dtype: float64

In [46]:
chargeoffs.cumsum()

0    0.1
1    0.2
2    0.3
3    0.4
4    0.5
5    0.5
dtype: float64

In [44]:
postloss_eop_bal = __get_postloss_eop_bal(
    preloss_eop_bal=preloss_eop_bal,
    chargeoffs=chargeoffs,
)
postloss_eop_bal

AssertionError: 0    0.9
1    0.8
2    0.7
3    0.6
4    0.5
5   -0.5
dtype: float64

In [38]:
postloss_eop_bal = _get_postloss_eop_bal(
    preloss_eop_bal=preloss_eop_bal,
    loss_rate=0.10,
)
postloss_eop_bal

0    0.9
1    0.9
2    0.9
3    0.9
4    0.9
5    0.0
dtype: float64

In [39]:
cpn_pmts = _get_cpn_pmts(
    bop_bal=postloss_eop_bal.shift(),
    cpn_frac=0.10,
)
cpn_pmts

0    0.00
1    0.09
2    0.09
3    0.09
4    0.09
5    0.09
dtype: float64